In [ ]:
#TODO: increase size
#TODO: change train to include labels, ignore for ae

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR='/home/evan/Datasets/tensorflow'
BATCH_SIZE=32
BUFFER_SIZE = 10000
IMG_SIZE = (64,64)

train_dataset = tfds.load('stanford_dogs', as_supervised=True, split='train', data_dir=DATA_DIR, download=False)
test_dataset = tfds.load('stanford_dogs', as_supervised=True, split='test', data_dir=DATA_DIR, download=False)

def transform(x , y):
    x = tf.image.resize(x, IMG_SIZE)
    x = tf.cast(x, tf.float32)
    x = (x - 127.5)/127.5
    return x, y
    
# train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.uint8), y))

def transform_ae(x,y):
    return (x,x,y)

test_dataset = test_dataset.map(lambda x, y : transform(x,y))

train_dataset = train_dataset.map(lambda x, y : transform(x,y))
ae_dataset = train_dataset.map(lambda x, y : transform_ae(x,y))
ae_dataset = ae_dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
print(train_dataset)
print(ae_dataset)
print(test_dataset)

In [ ]:
batch = next(train_dataset.as_numpy_iterator())[0]

In [ ]:
batch[1].sum()

In [ ]:
plt.imshow(((batch[1] *127.5)+127.5)/255.)

In [ ]:
[3]+[3]

In [5]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Dense, Conv2D, Conv2DTranspose , LeakyReLU, Dropout, Flatten, Input, Reshape

In [ ]:
#TODO: include option for use_bias = False
def Conv2DTransposeTied(Layer):
    def __init__(self, conv2D, activation=None, **kwargs):
        self.conv2D = conv2D
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias', initializer ='zeros'
                                      shape= [self.conv2D.input_shape[-1]]) #out_channels shape
    
    def call(self, inputs):
        Z = tf.matmul(inputs, self.conv2D.weights, transpose_b = True)
        return self.activation(Z + self.bases)

In [ ]:


EMBEDDED_DIM = 32

encoder_input_img = Input(IMG_SIZE+((3,)))
encoder_input_label = Input((1))
Z = Conv2D(64, (5, 5), strides=(1, 1), padding='same', input_shape=IMG_SIZE+((3,)))(encoder_input_img)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

Z = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

Z = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

Z = Flatten()(Z)
encoder_output_img = Dense(EMBEDDED_DIM)(Z)

In [ ]:
Z = Dense(8*8*256)(encoder_output_img)
Z = Reshape((8,8,256))(Z)

Z = Conv2D(128, (5, 5), strides=(1, 1), padding='same')(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

Z = Conv2DTranspose(64, (5,5), strides=(2,2), padding='same')(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

Z = Conv2DTranspose(64, (5,5), strides=(2,2), padding='same')(Z)
Z = LeakyReLU()(Z)
Z = Dropout(0.3)(Z)

decoder_output = Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same',activation=None)(Z)

autoencoder = Model(encoder_input_img, decoder_output)
autoencoder.output

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=.001)

autoencoder.compile(optimizer=optimizer, loss='mse')

In [ ]:
autoencoder.fit(ae_dataset, epochs=100, steps_per_epoch=100)

In [ ]:
# autoencoder.save('./autoencoder')

In [ ]:
import keras
autoencoder = keras.models.load_model('/home/evan/Desktop/Convolutional Random Forest/autoencoder')

In [ ]:
predictions = autoencoder.predict(train_dataset.take(1))
fig = plt.figure(figsize=(8,4))

for i in range(predictions.shape[0]):
    plt.subplot(4, 8, i+1)
    plt.imshow(((predictions[i, :, :, :]*127.5)+127.5)/255.)
    plt.axis('off')

In [ ]:
rf_dataset = train_dataset.map(lambda x,y : ((x,y),y))
encoder = keras.Model([encoder_input_img,encoder_input_label],[encoder_output_img, encoder_input_label])
print(rf_dataset)
rf_dataset = encoder.predict(rf_dataset, steps=1000) 

In [ ]:
rf_dataset[0].shape
rf_dataset[1].shape

In [ ]:
rf_image_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[0])
rf_label_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[1])
rf_combined_dataset = tf.data.Dataset.zip(((rf_image_dataset),(rf_label_dataset)))

In [ ]:
rf_combined_dataset

In [ ]:
import tensorflow_decision_forests as tfdf
from tensorflow.keras import Sequential
import keras

#run encoder to get embeddings
# (encoder_output)
rf_model = tfdf.keras.RandomForestModel()
rf_model.fit(rf_dataset[0],rf_dataset[1])

In [1]:
def run_inference(rf_model, encoder, input):
    embeddings = encoder.predict(input,tf.ones(input.shape[0]))
    return rf_model.predict(embeddings)
run_inference(rf_model, encoder, rf_dataset[0])

NameError: name 'rf_model' is not defined